In [1]:
import random
import polars as pl
from pathlib import Path
import json

In [2]:
nlg_ds = pl.read_csv('../../data/recipenlg/RecipeNLG_dataset.csv')#,  n_rows=1000)
nlg_ds = nlg_ds.with_columns(pl.col("ingredients").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.col("directions").str.json_decode())
nlg_ds = nlg_ds.with_columns(pl.arange(pl.len()).alias("index"))

In [3]:
import random
random.seed(123)
eval_data_idx = random.sample(range(0,nlg_ds.shape[0]), k=1000)

### The above code was used to generate data, but now since it is generated, we use the exact indexes in the generated data

In [4]:
with open("../../data/eval_data/eval_data.json", 'r') as f:
    read_idx_ds = pl.read_json(f)

In [5]:
eval_data_idx = list(read_idx_ds['index'])

In [6]:
out_ds = nlg_ds.filter(pl.col("index").is_in(eval_data_idx))

In [7]:
# Don't accidentally overwrite
#with open("../../data/eval_data/eval_data.json", "w") as f:
    #json.dump(out_ds.select('index', 'title', 'ingredients').to_dicts(), f, indent=2)

In [8]:
with open("../../data/eval_data/processed_data.json", "r") as f:
    processed_ds = pl.read_json(f)

In [9]:
processed_ds.head()

index,title,ingredients,google_search_query,main_food_items_and_preparations,important_ingredients_for_search,google_search_query_processed
i64,str,list[str],str,list[str],list[str],struct[2]
1461,"""Apricot Cake""","[""2 c. sugar"", ""4 eggs"", … ""1/2 c. chopped nuts (English or pecans)""]","""apricot cake recipe with baby …","[""sugar"", ""eggs"", … ""chopped nuts (English or pecans)""]","[""apricot baby food"", ""flour"", … ""Wesson oil""]","{""Apricot Cake"",[""baby food"", ""oil"", … ""cinnamon""]}"
1490,"""Angel Delights""","[""1 c. miniature marshmallows"", ""1 c. broken pretzels"", … ""1 lb. almond bark""]","""angel delights recipe with mar…","[""miniature marshmallows"", ""broken pretzels"", … ""almond bark""]","[""marshmallows"", ""pretzels"", … ""almond bark""]","{""Angel Delights"",[""marshmallows"", ""pretzels"", … ""almond bark""]}"
3374,"""Savory Sausage Casserole""","[""1 lb. bulk pork sausage"", ""1 c. uncooked rice"", … ""1/2 c. toasted halved or slivered blanched almonds""]","""sausage and rice casserole wit…","[""bulk pork sausage"", ""uncooked rice"", … ""toasted halved or slivered blanched almonds""]","[""pork sausage"", ""rice"", … ""almonds""]","{""Savory Sausage Casserole"",[""rice"", ""chicken noodle soup mix"", ""almonds""]}"
7006,"""Zemakove Knedliky A Kysele Zel…","[""2 c. mashed potatoes"", ""1/2 tsp. salt"", … ""1 1/2 c. flour""]","""potato dumplings with sauerkra…","[""mashed potatoes"", ""salt"", … ""flour""]","[""mashed potatoes"", ""sauerkraut"", … ""egg""]","{""Zemakove Knedliky A Kysele Zeli(Potato Dumplings With Sauerkraut) "",[""sauerkraut"", ""mashed potatoes""]}"
9957,"""Scalloped Sweet Potatoes""","[""1/4 c. sugar"", ""1/2 tsp. cinnamon"", … ""3 Tbsp. margarine""]","""scalloped sweet potatoes and a…","[""sugar"", ""cinnamon"", … ""margarine""]","[""sweet potatoes"", ""apples"", ""cinnamon""]","{""Scalloped Sweet Potatoes"",[""sweet potatoes"", ""apples""]}"


In [10]:
human_eval_data_idx = random.sample(eval_data_idx, k=50)

In [11]:
human_eval_ds = processed_ds.filter(pl.col("index").is_in(human_eval_data_idx))

In [12]:
human_eval_ds.head()

index,title,ingredients,google_search_query,main_food_items_and_preparations,important_ingredients_for_search,google_search_query_processed
i64,str,list[str],str,list[str],list[str],struct[2]
42580,"""Asparagus-Cucumber Mold""","[""1 (3 oz.) pkg. lime flavored gelatin"", ""1/2 c. boiling water"", … ""1/4 c. minced fresh parsley""]","""asparagus cucumber jello mold …","[""lime flavored gelatin"", ""boiling water"", … ""parsley (minced, fresh)""]","[""lime gelatin"", ""cream of asparagus soup"", … ""mayonnaise""]","{""Asparagus-Cucumber Mold"",[""jello"", ""cream of asparagus soup""]}"
89070,"""Picnic Beans""","[""1 (16 oz.) can kidney beans"", ""1 (16 oz.) can pork and beans"", … ""2 Tbsp. minced onion""]","""picnic beans recipe with bacon…","[""kidney beans"", ""pork and beans"", … ""minced onion""]","[""kidney beans"", ""pork and beans"", … ""bacon""]","{""Picnic Beans"",[""bacon"", ""kidney beans"", ""pork and beans""]}"
102340,"""Supreme Squash Casserole""","[""2 c. cooked and drained squash"", ""1 small onion, chopped"", … ""salt and pepper to taste""]","""squash casserole with sour cre…","[""squash (cooked, drained)"", ""onion (chopped)"", … ""salt and pepper""]","[""squash"", ""sour cream"", … ""Ritz crackers""]","{""Supreme Squash Casserole"",[""sour cream"", ""cream of chicken soup"", ""ritz crackers""]}"
165730,"""Jello Dessert (Or Green Goop)""","[""1 large Cool Whip"", ""1 large cottage cheese"", … ""chunk pineapple""]","""lime jello salad with cool whi…","[""Cool Whip"", ""cottage cheese"", … ""chunk pineapple""]","[""lime jello"", ""Cool Whip"", … ""pineapple""]","{""Jello Dessert (Or Green Goop)"",[""cool whip"", ""cottage cheese"", … ""pineapple""]}"
175496,"""Crepes""","[""6 eggs, well beaten"", ""1/2 tsp. salt"", … ""2 Tbsp. milk""]","""simple crepe recipe with eggs …","[""eggs (well beaten)"", ""salt"", … ""milk""]","[""eggs"", ""flour"", ""milk""]","{""Crepes"",[""eggs"", ""flour"", ""milk""]}"


In [13]:
# Don't accidentally overwrite
#with open("../../data/eval_data/human_annotations.json", "w") as f:
#    json.dump(human_eval_ds.select('index', 'title', 'ingredients').to_dicts(), f, indent=2)

In [14]:
with open("../../data/eval_data/human_annotations.json", "r") as f:
    human_annotations_ds = pl.read_json(f)

In [15]:
human_annotations_ds.head()

index,title,ingredients,human_ingredients,human_key_ingredients
i64,str,list[str],list[str],list[str]
25000,"""Pineapple Cream Cake""","[""1 pkg. white cake mix"", ""2 small pkg. vanilla instant pudding mix"", … ""4 oz. cream cheese""]","[""cake mix"", ""pudding mix"", … ""a can of pinapples""]","[""two packages of vanilla instant pudding mix"", ""one can of crushed pineapple""]"
135438,"""Pineapple Casserole""","[""2 large cans pineapple tidbits, drained"", ""3/4 c. flour"", … ""1 stick melted butter""]","[""canned pineapple"", ""flour"", … ""butter""]","[""grated cheddar cheese"", ""a pack of crushed ritz crackers""]"
230315,"""Low-Fat Chili""","[""1 1/2 lb. ground turkey"", ""1 1/2 c. chopped onion"", … ""3 c. brown rice, cooked""]","[""ground turkey"", ""onion"", … ""brown rice""]","[""coarsely chopped tomatoes"", ""cooked brown rice"", ""diced zucchini""]"
272288,"""Almond Bars""","[""1 c. butter"", ""2 c. flour"", ""1/2 c. powdered sugar""]","[""butter"", ""flour"", ""powdered sugar""]","[""butter"", ""flour"", ""powdered sugar""]"
275407,"""Pineapple Icebox Cake""","[""1 box yellow cake mix"", ""1 (20 oz.) can crushed pineapple (with juice)"", … ""1 c. chopped pecans""]","[""yellow cake mix"", ""can of pinapple"", … ""pecans""]","[""a box of yellow cake mix"", ""a carton of frozen whipped topping"", ""a can of pinapples""]"


In [16]:
human_eval_w_annotations_ds = human_eval_ds.with_columns(human_annotations_ds.select('human_ingredients'))
human_eval_w_annotations_ds = human_eval_w_annotations_ds.with_columns(human_annotations_ds.select('human_key_ingredients'))

In [17]:
human_eval_w_annotations_ds.head()

index,title,ingredients,google_search_query,main_food_items_and_preparations,important_ingredients_for_search,google_search_query_processed,human_ingredients,human_key_ingredients
i64,str,list[str],str,list[str],list[str],struct[2],list[str],list[str]
42580,"""Asparagus-Cucumber Mold""","[""1 (3 oz.) pkg. lime flavored gelatin"", ""1/2 c. boiling water"", … ""1/4 c. minced fresh parsley""]","""asparagus cucumber jello mold …","[""lime flavored gelatin"", ""boiling water"", … ""parsley (minced, fresh)""]","[""lime gelatin"", ""cream of asparagus soup"", … ""mayonnaise""]","{""Asparagus-Cucumber Mold"",[""jello"", ""cream of asparagus soup""]}","[""cake mix"", ""pudding mix"", … ""a can of pinapples""]","[""two packages of vanilla instant pudding mix"", ""one can of crushed pineapple""]"
89070,"""Picnic Beans""","[""1 (16 oz.) can kidney beans"", ""1 (16 oz.) can pork and beans"", … ""2 Tbsp. minced onion""]","""picnic beans recipe with bacon…","[""kidney beans"", ""pork and beans"", … ""minced onion""]","[""kidney beans"", ""pork and beans"", … ""bacon""]","{""Picnic Beans"",[""bacon"", ""kidney beans"", ""pork and beans""]}","[""canned pineapple"", ""flour"", … ""butter""]","[""grated cheddar cheese"", ""a pack of crushed ritz crackers""]"
102340,"""Supreme Squash Casserole""","[""2 c. cooked and drained squash"", ""1 small onion, chopped"", … ""salt and pepper to taste""]","""squash casserole with sour cre…","[""squash (cooked, drained)"", ""onion (chopped)"", … ""salt and pepper""]","[""squash"", ""sour cream"", … ""Ritz crackers""]","{""Supreme Squash Casserole"",[""sour cream"", ""cream of chicken soup"", ""ritz crackers""]}","[""ground turkey"", ""onion"", … ""brown rice""]","[""coarsely chopped tomatoes"", ""cooked brown rice"", ""diced zucchini""]"
165730,"""Jello Dessert (Or Green Goop)""","[""1 large Cool Whip"", ""1 large cottage cheese"", … ""chunk pineapple""]","""lime jello salad with cool whi…","[""Cool Whip"", ""cottage cheese"", … ""chunk pineapple""]","[""lime jello"", ""Cool Whip"", … ""pineapple""]","{""Jello Dessert (Or Green Goop)"",[""cool whip"", ""cottage cheese"", … ""pineapple""]}","[""butter"", ""flour"", ""powdered sugar""]","[""butter"", ""flour"", ""powdered sugar""]"
175496,"""Crepes""","[""6 eggs, well beaten"", ""1/2 tsp. salt"", … ""2 Tbsp. milk""]","""simple crepe recipe with eggs …","[""eggs (well beaten)"", ""salt"", … ""milk""]","[""eggs"", ""flour"", ""milk""]","{""Crepes"",[""eggs"", ""flour"", ""milk""]}","[""yellow cake mix"", ""can of pinapple"", … ""pecans""]","[""a box of yellow cake mix"", ""a carton of frozen whipped topping"", ""a can of pinapples""]"


In [18]:
# Don't accidentally overwrite
#with open("../../data/eval_data/human_annot_eval.json", "w") as f:
#    json.dump(human_eval_w_annotations_ds.to_dicts(), f, indent=2)

In [19]:
non_eval_idx = [i for i in range(nlg_ds.shape[0]) if i not in eval_data_idx]

In [20]:
eval_10k_idx = list(set(random.sample(non_eval_idx, k=9000) + eval_data_idx))

In [23]:
with open("../../data/eval_data/eval_10k.json", "r") as f:
    read_eval_10k_ds = pl.read_json(f)

In [24]:
eval_10k_idx = read_eval_10k_ds['index'].to_numpy()

In [25]:
non_eval_100k_idx = np.delete(np.arange(nlg_ds.shape[0]), eval_10k_idx)

In [26]:
non_eval_100k_idx

array([      0,       1,       2, ..., 2231139, 2231140, 2231141],
      shape=(2221142,))

In [29]:
eval_100k_idx = list(set(random.sample(list(non_eval_100k_idx), k=90000) + list(eval_10k_idx)))

In [30]:
len(eval_100k_idx)

100000

In [24]:
eval_10k_ds = nlg_ds.filter(pl.col('index').is_in(eval_10k_idx))

In [25]:
# Don't accidentally overwrite
#with open("../../data/eval_data/eval_10k.json", "w") as f:
#    json.dump(eval_10k_ds.to_dicts(), f, indent=2)

In [32]:
eval_100k_ds = nlg_ds.filter(pl.col('index').is_in(eval_100k_idx))

In [33]:
# Don't accidentally overwrite
#with open("../../data/eval_data/eval_100k.json", "w") as f:
#    json.dump(eval_100k_ds.to_dicts(), f, indent=2)

### Sanity check that we can read data properly

In [34]:
with open("../../data/eval_data/eval_100k.json", "r") as f:
    read_eval_100k_ds = pl.read_json(f)

In [35]:
(eval_100k_ds == read_eval_100k_ds).filter(pl.col('title') == False)

,title,ingredients,directions,link,source,NER,index
bool,bool,bool,bool,bool,bool,bool,bool


## Generate 150 datapoint for labeling that is not in any of our evaluation data

In [36]:
import numpy as np

In [37]:
eval_100k_idx = read_eval_100k_ds['index'].to_numpy()

In [38]:
eval_100k_idx

array([     64,      83,      88, ..., 2231086, 2231122, 2231137],
      shape=(100000,))

In [39]:
non_100k_idx = np.delete(np.arange(nlg_ds.shape[0]), eval_100k_idx)

In [40]:
np.arange(nlg_ds.shape[0])

array([      0,       1,       2, ..., 2231139, 2231140, 2231141],
      shape=(2231142,))

In [41]:
non_100k_idx

array([      0,       1,       2, ..., 2231139, 2231140, 2231141],
      shape=(2131142,))

In [46]:
label_150_idx = np.array(list(set(random.sample(list(non_100k_idx), k=150))))

In [50]:
label_150_idx.sort()

In [51]:
label_150_idx

array([  41254,   42099,   60699,   61742,   62233,   63247,   91437,
        109467,  146606,  150425,  150466,  185269,  192495,  196691,
        197970,  204596,  218859,  225014,  229259,  278795,  297220,
        317376,  319738,  346087,  364271,  371377,  375769,  379518,
        386200,  391990,  398235,  417520,  422987,  496661,  510940,
        526801,  552638,  580596,  589713,  594672,  600311,  600478,
        631618,  643177,  659740,  683934,  691275,  703046,  707281,
        730945,  735990,  764118,  801349,  820061,  823725,  826954,
        831303,  851295,  874570,  896740,  909353,  913607,  921543,
        923317,  937613,  972310,  991333, 1034874, 1042441, 1043965,
       1081875, 1112769, 1129356, 1158262, 1185614, 1189029, 1196358,
       1210700, 1214587, 1214662, 1221175, 1228520, 1231000, 1232842,
       1244642, 1252446, 1256357, 1259675, 1260661, 1265192, 1268011,
       1297789, 1304555, 1316825, 1352016, 1354643, 1358229, 1390880,
       1392526, 1405

In [52]:
label_150_ds = nlg_ds.filter(pl.col('index').is_in(label_150_idx))

In [54]:
i = 0
labels = []
for index, ingredients in label_150_ds['index', 'ingredients'].iter_rows():
    out_obj= {}
    text = "\n".join(ingredients)
    out_obj = {
        "data": {
            "id": index,
            "text": text
        },
    }
    labels.append(out_obj)

In [59]:
with open('../../data/NER_data/recipeNLG_to_label.json', mode='w') as f:
    json.dump(labels, f)

In [71]:
for i in label_150_ds.filter(pl.col('index') == 1297789)['ingredients'][0]:
    print(i)

2 pounds medium red-skinned potatoes
scrubbed Sea salt 1 pound skinless wild salmon fillet Safflower or sunflower oil
for greasing and frying Freshly ground pepper 1 bunch scallions (about 6 scallions)
coarsely chopped 3 large eggs
lightly beaten 3 garlic cloves
minced 2 tablespoons minced fresh ginger 1/2 medium red onion
finely chopped 1 tablespoon tamari 1 tablespoon toasted sesame oil 1/2 cup plain dry bread crumbs Dill Sauce
for serving
